In [15]:
import pandas as pd
import numpy as np
import seaborn as sns;sns.set()
print(sns.__version__)
import matplotlib as plt
import pickle

0.9.0


In [16]:
df = pd.read_csv("peds5.csv")
df.head()

,M,F,Difference,Speed,Distance,Lane 1,Lane 2,Lane 3,2W,4W,Heavy,Accept
0,1,0,1.115,17.937220,40,1,0,0,0,1,0,0
1,1,0,1.071,18.674136,15,1,0,0,0,1,0,0
2,1,0,1.658,12.062726,28,0,1,0,0,0,1,0
3,1,0,1.039,19.249278,15,0,1,0,0,1,0,0
4,1,0,1.219,16.406891,42,1,0,0,0,0,1,0


In [0]:
with open("train_indices.pickle",'rb') as handle:
    training_indices = pickle.load(handle)
df_train = df.iloc[training_indices,:]
del df
del training_indices

In [20]:
df_train = df_train.drop(['Difference'],axis=1)
df_train.head()

,M,F,Speed,Distance,Lane 1,Lane 2,Lane 3,2W,4W,Heavy,Accept
145,1,1,18.484288,20,0,1,0,0,1,0,0
548,2,0,15.467904,8,1,0,0,0,1,0,0
76,1,0,18.921476,10,0,1,0,0,1,0,0
381,1,0,19.342360,27,1,0,0,0,1,0,0
516,2,0,16.142050,10,0,0,1,0,0,1,0


In [21]:
X_train = df_train.iloc[:,:-1].values
Y_train = df_train.iloc[:,-1].values
s = 2
d = 3

print(X_train.shape)
print(Y_train.shape)

(774, 10)
(774,)


In [25]:
list(X_train[0:2])[0]

array([ 1.        ,  1.        , 18.48428835, 20.        ,  0.        ,
        1.        ,  0.        ,  0.        ,  1.        ,  0.        ])

In [0]:
tc = 0
c = 0

In [0]:
def condition(X,y):

    # Check if points lie in correct region

    coefficients = np.array([ 0.10480379,-0.20240534])
    intercept = -3.91229859

    if y == 0:
        y = -1

    return y*(coefficients[0]*X[s]+coefficients[1]*X[d]+intercept) > 0

def conditional_resampling(X,y):

# Pass numpy arrays as input

    from imblearn.over_sampling import SMOTE

    sm = SMOTE(random_state=2)
    print(f"Shapes before resampling: X{np.array(X).shape}, Y{np.array(y).shape}")

    X_res,y_res = sm.fit_resample(np.array(X), np.array(y).ravel())
    y_res = np.reshape(y_res,(y_res.shape[0],1))

    data_cres = [(X_res[i],y_res[i]) for i in range(len(y_res)) if (X_res[i].tolist() in X.tolist()) or condition(X_res[i],y_res[i]) ]

    print(len(data_cres),len(X_res))
                                                               
    return np.array([x[0] for x in data_cres]), np.array([x[1] for x in data_cres])


In [52]:
Xres, yres = conditional_resampling(X_train,Y_train)
print(Xres.shape,yres.shape)

Shapes before resampling: X(774, 10), Y(774,)
1438 1438
(1438, 10) (1438, 1)
